In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com//oauth2/token"}


dbutils.fs.mount(
source = "abfss://tokyo-olympic-data@jmstokyoolympic.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/tokyoolympic",
extra_configs = configs)

True

In [0]:
%fs
ls "mnt/tokyoolympic"

path,name,size,modificationTime
dbfs:/mnt/tokyoolympic/raw-data/,raw-data/,0,1719823639000
dbfs:/mnt/tokyoolympic/transformed-data/,transformed-data/,0,1719823646000


In [0]:
Athletes = spark.read.format('csv').option('header','true').load('/mnt/tokyoolympic/raw-data/Athletes.csv')
Coaches = spark.read.format('csv').option('header','true').load('/mnt/tokyoolympic/raw-data/Coaches.csv')
EntriesGender = spark.read.format('csv').option('header','true').load('/mnt/tokyoolympic/raw-data/EntriesGender.csv')
Medals = spark.read.format('csv').option('header','true').load('/mnt/tokyoolympic/raw-data/Medals.csv')
Teams = spark.read.format('csv').option('header','true').load('/mnt/tokyoolympic/raw-data/Teams.csv')

In [0]:
Athletes.show(5)
Coaches.show(5)
EntriesGender.show(5)
Medals.show(5)
Teams.show(5)

+-----------------+-------+-------------------+
|       PersonName|Country|         Discipline|
+-----------------+-------+-------------------+
|  AALERUD Katrine| Norway|       Cycling Road|
|      ABAD Nestor|  Spain|Artistic Gymnastics|
|ABAGNALE Giovanni|  Italy|             Rowing|
|   ABALDE Alberto|  Spain|         Basketball|
|    ABALDE Tamara|  Spain|         Basketball|
+-----------------+-------+-------------------+
only showing top 5 rows

+---------------+-------------+----------+-----+
|           Name|      Country|Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|        Egypt|  Football| NULL|
|      ABE Junya|        Japan|Volleyball| NULL|
|  ABE Katsuhiko|        Japan|Basketball| NULL|
|   ADAMA Cherif|C�te d'Ivoire|  Football| NULL|
|     AGEBA Yuya|        Japan|Volleyball| NULL|
+---------------+-------------+----------+-----+
only showing top 5 rows

+-------------------+------+----+-----+
|         Discipline|Female|Male|Tota

In [0]:
Athletes.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [0]:
Coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
EntriesGender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)



In [0]:
EntriesGender = EntriesGender.withColumn('Female',col('Female').cast(IntegerType()))\
    .withColumn('Male',col('Male').cast(IntegerType()))\
        .withColumn('Total',col('Total').cast(IntegerType()))

In [0]:
EntriesGender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
Medals.printSchema()

root
 |-- Rank: string (nullable = true)
 |-- Team_Country: string (nullable = true)
 |-- Gold: string (nullable = true)
 |-- Silver: string (nullable = true)
 |-- Bronze: string (nullable = true)
 |-- Total: string (nullable = true)
 |-- Rank by Total: string (nullable = true)



In [0]:
Medals = Medals.withColumn('Rank',col('Rank').cast(IntegerType()))\
    .withColumn('Gold',col('Gold').cast(IntegerType()))\
        .withColumn('Silver',col('Silver').cast(IntegerType()))\
            .withColumn('Bronze',col('Bronze').cast(IntegerType()))\
                .withColumn('Total',col('Total').cast(IntegerType()))

In [0]:
Medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Team_Country: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: string (nullable = true)



In [0]:
Teams.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
Athletes.repartition(1).write.mode('overwrite').option('header','true').csv('/mnt/tokyoolympic/transformed-data/Athletes')

In [0]:
Coaches.repartition(1).write.mode('overwrite').option('header','true').csv('/mnt/tokyoolympic/transformed-data/Coaches')
EntriesGender.repartition(1).write.mode('overwrite').option('header','true').csv('/mnt/tokyoolympic/transformed-data/EntriesGender')
Medals.repartition(1).write.mode('overwrite').option('header','true').csv('/mnt/tokyoolympic/transformed-data/Medals')
Teams.repartition(1).write.mode('overwrite').option('header','true').csv('/mnt/tokyoolympic/transformed-data/Teams')